# MeshCat Animations

MeshCat.jl also provides an animation interface, built on top of the [three.js animation system](https://threejs.org/docs/#manual/introduction/Animation-system). While it is possible to construct animation clips and tracks manually, just as you would in Three.js, it's generally easier to use the MeshCat `Animation` type.

Let's show off building a simple animation. We first have to create our scene: 

In [1]:
# Optional: 
# These commands tell the Julia package manager to use the exact
# set of dependencies specified in the Project.toml file in this folder. 
# That should give you a nice, reproducible environment for testing. 

using Pkg
Pkg.activate(@__DIR__)
Pkg.develop(Pkg.PackageSpec(path=dirname(@__DIR__)))
Pkg.instantiate()

 Activating environment at `d:\Users\Personal\Personal\Projects\GitRepo\LearnJulia\Ex\Project.toml`


LoadError: could not find project file in package at `d:\Users\Personal\Personal\Projects\GitRepo\LearnJulia` maybe `subdir` needs to be specified

In [4]:
using MeshCat, GeometryBasics, CoordinateTransformations

In [5]:
vis = Visualizer()

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8702
└ @ MeshCat C:\Users\dengy\.julia\packages\MeshCat\DjQfQ\src\visualizer.jl:73


MeshCat Visualizer with path /meshcat at http://127.0.0.1:8702

In [15]:
## To open the visualizer in a new browser tab, do: 
# open(vis)

## To open the visualizer inside this jupyter notebook, do: 
# render(vis)

## To open this visualizer in a standalone window, do:
# using Blink
# AtomShell.isinstalled() || AtomShell.install()
# open(vis, Window())

open(vis)

Process(`cmd.exe /C 'start http://127.0.0.1:8702'`, ProcessExited(0))

In [16]:
setobject!(vis[:box1], 
    Rect(Vec(0., 0, 0), Vec(0.1, 0.2, 0.3)))

MeshCat Visualizer with path /meshcat/box1 at http://127.0.0.1:8702

### Building an Animation

We construct an animation by first creating a blank `Animation()` object. We can then use the `atframe` function to set properties or transforms of the animation at specific frames of the animation. Three.js will automatically interpolate between whatever values we provide. 

For example, let's animate moving the box from [0, 0, 0] to [0, 1, 0]: 

In [17]:
anim = Animation()

atframe(anim, 0) do
    # within the context of atframe, calls to 
    # `settransform!` and `setprop!` are intercepted
    # and recorded in `anim` instead of having any
    # effect on `vis`.
    settransform!(vis[:box1], Translation(0., 0, 0))
end

atframe(anim, 30) do
    settransform!(vis[:box1], Translation(0., 1, 0))
end

# `setanimation!()` actually sends the animation to the
# viewer. By default, the viewer will play the animation
# right away. To avoid that, you can also pass `play=false`. 
setanimation!(vis, anim)

You should see the box slide 1 meter to the right in the viewer. If you missed the animation, you can run it again from the viewer. Click "Open Controls", find the "Animations" section, and click "play". 

### Animating the Camera

The camera is just another object in the MeshCat scene. To set its transform, we just need to index into the visualizer with the right path (note the leading `/`):

In [18]:
settransform!(vis["/Cameras/default"], Translation(0, 0, 1))

MeshCat Visualizer with path /Cameras/default at http://127.0.0.1:8702

To animate the camera, we just have to do that same kind of `settransform!` to individual frames in an animation: 

In [19]:
anim = Animation()

atframe(anim, 0) do
    settransform!(vis["/Cameras/default"], Translation(0., 0, 0))
end

atframe(anim, 30) do
    settransform!(vis["/Cameras/default"], Translation(0., 0, 1))
end
 
setanimation!(vis, anim)

We can also animate object properties. For example, let's animate the camera's `zoom` property to smoothly zoom out and then back in. Note that to do this, we have to access a deeper path in the visualizer to get to the actual camera object. For more information, see: https://github.com/rdeits/meshcat#camera-control

In [20]:
anim = Animation()

atframe(anim, 0) do
    setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1)
end

atframe(anim, 30) do
    setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 0.5)
end

atframe(anim, 60) do
    setprop!(vis["/Cameras/default/rotated/<object>"], "zoom", 1)
end
 
setanimation!(vis, anim)

## Making Objects Appear and Disappear

MeshCat does not support adding or deleting objects in frames of an animation. Instead, you will need to call `setobject!` ahead of time to add all the objects in the scene, then call `setvisible!` on individual frames to toggle the visibility of objects on and off. 

Let's demonstrate this by animating a moving box and a sphere which appears halfway through the animation:

In [21]:
setobject!(vis[:box1], 
    Rect(Vec(0., 0, 0), Vec(0.1, 0.2, 0.3)))
setobject!(vis[:sphere],
    HyperSphere(Point(0., 0, 0), 0.25))

# Start with the sphere hidden
setvisible!(vis[:sphere], false)

MeshCat Visualizer with path /meshcat/sphere at http://127.0.0.1:8702

In [22]:
anim = Animation()
atframe(anim, 0) do
    setvisible!(vis[:sphere], false)
    settransform!(vis[:box1], Translation(0., 0, 0))
end

atframe(anim, 15) do
    setvisible!(vis[:sphere], true)
end

atframe(anim, 30) do
    settransform!(vis[:box1], Translation(0., 1, 0))
end
   
setanimation!(vis, anim)

### Recording an Animation

To record an animation at a smooth, fixed frame rate, click on "Open Controls" in the viewer, and then go to "Animations" -> "default" -> "Recording" -> "record". This will play the entire animation, recording every frame and then let you download the resulting frames to your computer. 

To record activity in the MeshCat window that isn't a MeshCat animation, we suggest using a screen-capture tool like Quicktime for macOS or RecordMyDesktop for Linux. 

### Converting the Animation into a Video

Currently, meshcat can only save an animation as a `.tar` file consisting of a  list of `.png` images, one for each frame. To convert that into a video, you can run: 

In [23]:
# MeshCat.convert_frames_to_video(
#     "/home/rdeits/Downloads/meshcat_1528401494656.tar")